In [1]:
# Load packagesfrom segment_anything import SamPredictor, sam_model_registry
from segment_anything import SamPredictor, sam_model_registry
from torch.utils.data import DataLoader
from torchvision import transforms
from PIL import Image
import torch
import os
import torch
from segment_anything import sam_model_registry
import cv2
from segment_anything import SamAutomaticMaskGenerator
import pickle
import numpy as np



In [4]:
# Loading the Segment Anything Model
DEVICE = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
MODEL_TYPE = "vit_h"
CHECKPOINT_PATH="/Users/nirwantandukar/Documents/trial_BZea/sam_vit_h_4b8939.pth"
sam = sam_model_registry[MODEL_TYPE](checkpoint=CHECKPOINT_PATH)
sam.to(device=DEVICE)



Sam(
  (image_encoder): ImageEncoderViT(
    (patch_embed): PatchEmbed(
      (proj): Conv2d(3, 1280, kernel_size=(16, 16), stride=(16, 16))
    )
    (blocks): ModuleList(
      (0-31): 32 x Block(
        (norm1): LayerNorm((1280,), eps=1e-06, elementwise_affine=True)
        (attn): Attention(
          (qkv): Linear(in_features=1280, out_features=3840, bias=True)
          (proj): Linear(in_features=1280, out_features=1280, bias=True)
        )
        (norm2): LayerNorm((1280,), eps=1e-06, elementwise_affine=True)
        (mlp): MLPBlock(
          (lin1): Linear(in_features=1280, out_features=5120, bias=True)
          (lin2): Linear(in_features=5120, out_features=1280, bias=True)
          (act): GELU(approximate='none')
        )
      )
    )
    (neck): Sequential(
      (0): Conv2d(1280, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (1): LayerNorm2d()
      (2): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (3): LayerNorm2d

In [3]:
# Automated Mask (Instance Segmentation) Generation with SAM
# segmentation - [np.ndarray] - the mask with (W, H) shape, and bool type, where W and H are the width and height 
#of the original image, respectively
#area - [int] - the area of the mask in pixels
#bbox - [List[int]] - the boundary box detection in xywh format
#predicted_iou - [float] - the model's own prediction for the quality of the mask
#point_coords - [List[List[float]]] - the sampled input point that generated this mask
#stability_score - [float] - an additional measure of mask quality
#crop_box - List[int] - the crop of the image used to generate this mask in xywh format

#mask_generator = SamAutomaticMaskGenerator(sam)
IMAGE_PATH = "maize.png"
image_bgr = cv2.imread(IMAGE_PATH)
image_rgb = cv2.cvtColor(image_bgr, cv2.COLOR_BGR2RGB)



[ WARN:0@9.014] global loadsave.cpp:248 findDecoder imread_('maize.png'): can't open/read file: check file path/integrity


error: OpenCV(4.8.0) /Users/runner/work/opencv-python/opencv-python/opencv/modules/imgproc/src/color.cpp:182: error: (-215:Assertion failed) !_src.empty() in function 'cvtColor'


In [4]:
# Load the image
image = cv2.imread('maize.jpeg')



In [5]:
# 1. Image Compression using Gaussian Pyramid
compressed_image = cv2.pyrDown(image)
cv2.imshow('Processed Image', compressed_image)


In [6]:
# 2. Background Separation
# Convert to LAB color space
lab_image = cv2.cvtColor(compressed_image, cv2.COLOR_BGR2Lab)
l_channel, a_channel, b_channel = cv2.split(lab_image)

# Thresholding on the blue channel to segment maize kernels
_, thresh = cv2.threshold(b_channel, 128, 255, cv2.THRESH_BINARY_INV)

cv2.imshow('Processed Image', lab_image)

In [7]:
# 3. Mean Shift Filtering
# Parameters for mean shift
sp = 40  # spatial window radius
sr = 60  # color window radius
mean_shift_filtered = cv2.pyrMeanShiftFiltering(compressed_image, sp, sr, maxLevel=3)
cv2.imshow('Processed Image', mean_shift_filtered)


In [8]:
# 4. Enhancement of Kernel Edges using Colour Deconvolution
# Convert to LAB color space
lab_filtered = cv2.cvtColor(mean_shift_filtered, cv2.COLOR_BGR2Lab)
l_channel, a_channel, b_channel = cv2.split(lab_filtered)

# Define OD matrices for the three stain combinations
methyl_green = np.array([0.98003, 0.144316, 0.133146])
hematoxylin_gl = np.array([0.644211, 0.716556, 0.266844])
white = np.array([0, 0, 0])

# Choose the appropriate stain combination based on the average LAB values
avg_l = np.mean(l_channel)
avg_a = np.mean(a_channel)
avg_b = np.mean(b_channel)

# Calculate Euclidean distances for each stain combination
dist_methyl_green = np.linalg.norm(methyl_green - np.array([avg_l, avg_a, avg_b]))
dist_hematoxylin_gl = np.linalg.norm(hematoxylin_gl - np.array([avg_l, avg_a, avg_b]))
dist_white = np.linalg.norm(white - np.array([avg_l, avg_a, avg_b]))

# Choose the stain with the minimum distance
if min(dist_methyl_green, dist_hematoxylin_gl, dist_white) == dist_methyl_green:
    chosen_stain = methyl_green
elif min(dist_methyl_green, dist_hematoxylin_gl, dist_white) == dist_hematoxylin_gl:
    chosen_stain = hematoxylin_gl
else:
    chosen_stain = white

# Apply Colour Deconvolution
# Convert the LAB image to a linear scale
linear_lab = np.exp(-1.0 * lab_filtered / 255.0)
# Apply the chosen stain
deconvoluted = linear_lab * chosen_stain

# Convert back to 8-bit image
deconvoluted = (deconvoluted * 255).astype(np.uint8)

# Display the processed image
#cv2.imshow('Processed Image', deconvoluted)
#cv2.waitKey(0)
#cv2.destroyAllWindows()

In [9]:
# Kernel Segmentation
# Convert the processed image to grayscale
gray_image = cv2.cvtColor(deconvoluted, cv2.COLOR_BGR2GRAY)

# Otsu's thresholding
_, otsu_thresh = cv2.threshold(gray_image, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)

# Adaptive thresholding
block_size = 25  # This should be set to the average length of kernels MBR
adaptive_thresh = cv2.adaptiveThreshold(gray_image, 255, cv2.ADAPTIVE_THRESH_MEAN_C, cv2.THRESH_BINARY, block_size, 2)

# Kernel Recognition
# Step 1: Smoothing the kernel binary image with Gaussian filter
kernel_size = 5  # This should be set to the average width of seed MBR
smoothed_image = cv2.GaussianBlur(adaptive_thresh, (kernel_size, kernel_size), 0)

# Step 2: Local maximum detection
local_maxima = cv2.dilate(smoothed_image, np.ones((kernel_size, kernel_size), np.uint8))
local_maxima = cv2.compare(smoothed_image, local_maxima, cv2.CMP_EQ)

# Step 3: Eliminate spurious kernel recognition points
# Find contours in the local_maxima image
contours, _ = cv2.findContours(local_maxima, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
for contour in contours:
    if cv2.contourArea(contour) < 5:  # This threshold can be adjusted
        cv2.drawContours(local_maxima, [contour], 0, 0, -1)

# Display the results
cv2.imshow('Otsu Thresholding', otsu_thresh)
cv2.imshow('Adaptive Thresholding', adaptive_thresh)
cv2.imshow('Kernel Recognition', local_maxima)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [ ]:
# Assuming DPI is 300 (change this to your actual DPI)
dpi = 144
# Calculate the number of pixels to trim from each side
trim_pixels = 1 * dpi
# Trim the image
trimmed_image_rgb = image_rgb[trim_pixels:-trim_pixels, trim_pixels:-trim_pixels]

# Save the trimmed image if you want
#cv2.imwrite('/path/to/save/trimmed_image.jpg', cv2.cvtColor(trimmed_image_rgb, cv2.COLOR_RGB2BGR))


result = mask_generator.generate(trimmed_image_rgb)

# Sort the result by area in descending order
sorted_result = sorted(result, key=lambda x: x['area'], reverse=True)

# Take the top 20 largest area masks
top_20_result = sorted_result[:30]

# Iterate through the list of top 20 segments
for i, segment in enumerate(top_20_result):
    # Convert boolean mask to uint8
    mask_uint8 = np.uint8(segment['segmentation'] * 255)
    
    # Show the mask
    cv2.imshow(f'Top 20 Generated Mask {i+1}', mask_uint8)
    cv2.waitKey(0)

cv2.destroyAllWindows()